<a href="https://colab.research.google.com/github/ileri-oluwa-kiiye/cs231n/blob/main/knn_cifar10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as plt
import numpy as np
%matplotlib inline
pd.plotting.register_matplotlib_converters()
print("Setup Complete")

Setup Complete


In [2]:
#Loading the cifar-10 dataset
import tensorflow as tf

(X_train, y_train), (X_test, y_test) = tf.keras.datasets.cifar10.load_data()

X_train.shape

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


(50000, 32, 32, 3)

In [3]:
X_test.shape

(10000, 32, 32, 3)

In [4]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

#For the data preprocessing
#Reshaping the data to be two dimensional
def preprocess_data(X_train, X_test):
  X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
  X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

  #using the standard scaler to enable a more uniform data
  scaler = StandardScaler()
  X_train_scaled = scaler.fit_transform(X_train_reshaped)
  X_test_scaled = scaler.transform(X_test_reshaped)

  return X_train_scaled, X_test_scaled


In [5]:
#Defining a parameter grid
param_grid = {
    'n_neighbors': [ 3, 5, 7, 9],  # different k values to try
    'metric': ['manhattan', 'euclidean', 'minkowski'],  # different distance metrics
    'weights': ['uniform', 'distance']  # weight function used in prediction
}


# Creating the KNN Classifier
knn = KNeighborsClassifier()
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)


In [6]:
# Since the cifar dataset is large, let's use a part of it to perform the grid search
train_subset_size = 5000
test_subset_size = 1000

X_train_subset = X_train[:train_subset_size]
y_train_subset = y_train[:train_subset_size]

X_test_subset = X_test[:test_subset_size]
y_test_subset = y_test[:test_subset_size]

X_train_preprocessed, X_test_preprocessed = preprocess_data(X_train_subset, X_test_subset)

In [7]:
#grid search
grid_search = GridSearchCV(
    knn,
    param_grid,
    cv= cv,
    n_jobs = 1,
    verbose = 1,
    scoring = 'accuracy',
    return_train_score=True,
)

grid_search.fit(X_train_preprocessed, y_train_subset.ravel())
print("\nBest params:", grid_search.best_params_)
print("\nBest Cross Validation Score:", grid_search.best_score_)

Fitting 5 folds for each of 24 candidates, totalling 120 fits

Best params: {'metric': 'manhattan', 'n_neighbors': 9, 'weights': 'distance'}

Best Cross Validation Score: 0.31839999999999996


In [8]:
#get the best model and make predictions
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test_preprocessed)


# Print detailed evaluation metrics
print("\nDetailed Classification Report:")
print(classification_report(y_test_subset, y_pred))

# Print confusion matrix
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_subset, y_pred))

# Print top 3 best parameter combinations
print("\nTop 3 parameter combinations:")
results_df = pd.DataFrame(grid_search.cv_results_)
top_3 = results_df.nlargest(3, 'mean_test_score')
for idx, row in top_3.iterrows():
    print(f"\nRank {idx + 1}:")
    print(f"Parameters: {row['params']}")
    print(f"Mean CV Score: {row['mean_test_score']:.4f} (+/- {row['std_test_score']*2:.4f})")


Detailed Classification Report:
              precision    recall  f1-score   support

           0       0.39      0.46      0.42       103
           1       0.92      0.13      0.24        89
           2       0.26      0.50      0.34       100
           3       0.33      0.16      0.21       103
           4       0.18      0.42      0.25        90
           5       0.29      0.14      0.19        86
           6       0.25      0.24      0.24       112
           7       0.55      0.22      0.31       102
           8       0.42      0.72      0.53       106
           9       0.69      0.23      0.34       109

    accuracy                           0.33      1000
   macro avg       0.43      0.32      0.31      1000
weighted avg       0.43      0.33      0.31      1000


Confusion Matrix:
[[47  0 12  2  9  1  5  1 25  1]
 [ 8 12 12  4 15  1 10  0 23  4]
 [15  0 50  1 18  3  7  1  5  0]
 [ 9  1 21 16 22 11 18  1  4  0]
 [ 7  0 20  5 38  1  8  2  8  1]
 [ 6  0 19  6 22 12 11  